In [1]:
import os
import torch
import torch.nn as nn
from torchvision.models.video import r3d_18
from monai.networks.nets import DenseNet121

2025-07-12 21:08:55.950097: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752354536.462509   21104 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752354536.602302   21104 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752354537.700043   21104 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752354537.700094   21104 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752354537.700099   21104 computation_placer.cc:177] computation placer alr

In [2]:
def get_resnet3d_model(n_classes):
    model = r3d_18(pretrained=False)
    model.stem[0] = nn.Conv3d(
        1, 64, kernel_size=(3,7,7), stride=(1,2,2), padding=(1,3,3), bias=False
    )
    model.fc = nn.Linear(model.fc.in_features, n_classes)
    return model

def get_densenet3d_model(n_classes):
    return DenseNet121(
        spatial_dims=3,
        in_channels=1,
        out_channels=n_classes
    )

def identify_model_type(checkpoint_path, n_classes=2):
    try:
        checkpoint = torch.load(checkpoint_path, map_location='cpu')
    except Exception as e:
        print(f" Error cargando {checkpoint_path}: {e}")
        return "ERROR"

    # A veces guardaste el modelo completo
    if not isinstance(checkpoint, dict):
        print(f" {os.path.basename(checkpoint_path)} parece contener el modelo completo")
        print(f"Tipo: {type(checkpoint)}")
        return str(type(checkpoint))

    # Intentar cargar en ResNet3D
    try:
        resnet_model = get_resnet3d_model(n_classes)
        resnet_model.load_state_dict(checkpoint, strict=True)
        return "ResNet3D (r3d_18)"
    except Exception:
        pass

    # Intentar cargar en DenseNet121 3D
    try:
        densenet_model = get_densenet3d_model(n_classes)
        densenet_model.load_state_dict(checkpoint, strict=True)
        return "DenseNet3D (DenseNet121)"
    except Exception:
        pass

    return "Desconocido"

def analyze_folder(models_folder):
    print(f"Analizando modelos en carpeta: {models_folder}")
    model_files = [f for f in os.listdir(models_folder) if f.endswith('.pth')]

    if not model_files:
        print(" No se encontraron archivos .pth")
        return

    for file in sorted(model_files):
        path = os.path.join(models_folder, file)
        print(f"\n Archivo: {file}")

        result = identify_model_type(path)
        print(f" Detectado: {result}")

analyze_folder("/mnt/homeGPU/mcribilles/TFG/modelos_con_pesos/modelos_preentrenados/pretrainings")


Analizando modelos en carpeta: /mnt/homeGPU/mcribilles/TFG/modelos_con_pesos/modelos_preentrenados/pretrainings

 Archivo: best_model_gridsearch_adrenalmnist3d_64_ceL_stop_acc.pth


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
Arguments other than a weigh

 Detectado: ResNet3D (r3d_18)

 Archivo: best_model_gridsearch_adrenalmnist3d_64_ceL_stop_loss.pth
 Detectado: ResNet3D (r3d_18)

 Archivo: best_model_gridsearch_adrenalmnist3d_64_ceLcontrastiveL_stop_acc.pth
 Detectado: ResNet3D (r3d_18)

 Archivo: best_model_gridsearch_adrenalmnist3d_64_ceLcontrastiveL_stop_loss.pth
 Detectado: ResNet3D (r3d_18)

 Archivo: best_model_gridsearch_adrenalmnist3d_64_ceLtripletL_stop_acc.pth
 Detectado: ResNet3D (r3d_18)

 Archivo: best_model_gridsearch_adrenalmnist3d_64_ceLtripletL_stop_loss.pth
 Detectado: ResNet3D (r3d_18)

 Archivo: best_model_gridsearch_adrenalmnist3d_64_ceLtripletLcontrastiveL_stop_acc.pth
 Detectado: ResNet3D (r3d_18)

 Archivo: best_model_gridsearch_adrenalmnist3d_64_ceLtripletLcontrastiveL_stop_loss.pth
 Detectado: ResNet3D (r3d_18)

 Archivo: best_model_gridsearch_adrenalmnist3d_64_tripletL_stop_loss.pth
 Detectado: ResNet3D (r3d_18)

 Archivo: best_model_gridsearch_nodulemnist3d_64_ceL_stop_acc.pth
 Detectado: ResNet3D (r3d_18)

In [ ]:
checkpoint = torch.load("/mnt/homeGPU/mcribilles/TFG/pretrainings/best_model_gridsearch_organmnist3d_64_ceL_stop_acc.pth", map_location='cpu')
print(type(checkpoint))
print(checkpoint.keys() if isinstance(checkpoint, dict) else "No es dict")

<class 'collections.OrderedDict'>
odict_keys(['stem.0.weight', 'stem.1.weight', 'stem.1.bias', 'stem.1.running_mean', 'stem.1.running_var', 'stem.1.num_batches_tracked', 'layer1.0.conv1.0.weight', 'layer1.0.conv1.1.weight', 'layer1.0.conv1.1.bias', 'layer1.0.conv1.1.running_mean', 'layer1.0.conv1.1.running_var', 'layer1.0.conv1.1.num_batches_tracked', 'layer1.0.conv2.0.weight', 'layer1.0.conv2.1.weight', 'layer1.0.conv2.1.bias', 'layer1.0.conv2.1.running_mean', 'layer1.0.conv2.1.running_var', 'layer1.0.conv2.1.num_batches_tracked', 'layer1.1.conv1.0.weight', 'layer1.1.conv1.1.weight', 'layer1.1.conv1.1.bias', 'layer1.1.conv1.1.running_mean', 'layer1.1.conv1.1.running_var', 'layer1.1.conv1.1.num_batches_tracked', 'layer1.1.conv2.0.weight', 'layer1.1.conv2.1.weight', 'layer1.1.conv2.1.bias', 'layer1.1.conv2.1.running_mean', 'layer1.1.conv2.1.running_var', 'layer1.1.conv2.1.num_batches_tracked', 'layer2.0.conv1.0.weight', 'layer2.0.conv1.1.weight', 'layer2.0.conv1.1.bias', 'layer2.0.conv1

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
